In [1]:
import os 
import json
import requests
import pandas as pd
from io import StringIO
from time import sleep
from random import randint
from datetime import datetime, timedelta

In [3]:
def toNumeric(x):
    try:
        return float(str(x).replace(',', ''))
    except:
        return None
    
def strToFloat(pdf, num_cols):
    tmp = pdf.copy()
    for col in num_cols:
        tmp.loc[:, col] = tmp[col].apply(toNumeric)
    return tmp

def toRocYear(date_str, sep='/'):
    YEAR_OFFSET = 1911
    split = date_str.split(sep)
    ori_year = split[0]
    new_year = str(int(ori_year) - YEAR_OFFSET)
    new_date_str = date_str.replace(ori_year, new_year)
    return new_date_str

now = datetime.now()
start_dt = now.strftime('%Y/%m/%d')
end_dt = now.strftime('%Y/%m/%d')

output_path = "/Users/fang/stock_data/basic_data"
# output_path = "C:\\Users\\Fang\\PycharmProjects\\股票分析\\stock_data\\basic_data"
# output_path = "C:\\Users\\Flora\\Documents\\股票分析\\stock_data\\basic_data"

if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)

dt = datetime.strptime(start_dt, '%Y/%m/%d')
end_dt = datetime.strptime(end_dt, '%Y/%m/%d')

tpex_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&d=%s'
twse_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=%s&type=ALL'
tpex_header = ['股票代號', '名稱', '收盤價', '漲跌價差', '開盤價', '最高價', '最低價', '均價', '成交股數', '成交金額', 
               '成交筆數', '最後買價', '最後買量', '最後賣價', '最後賣量', '發行股數', '次日參考價', '次日漲停價', '次日跌停價']
header_list = ['股票代號', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
header_str = '股票代號,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,漲跌幅,成交筆數\n'
while dt <= end_dt:
    if dt.weekday() in range(0, 5):
        print(dt.strftime('%Y/%m/%d'))
        
        roc_dt = toRocYear(dt.strftime('%Y/%m/%d'))
        tpex_resp = requests.get(tpex_url % roc_dt)
        json_obj = json.loads(tpex_resp.text)
        aaData = json_obj['aaData']
        tpex_pdf = pd.DataFrame(aaData, columns=tpex_header)
        tpex_pdf = tpex_pdf[tpex_pdf['股票代號'].str.len() == 4]
        tpex_pdf = tpex_pdf[header_list]
        
        twse_resp = requests.get(twse_url % dt.strftime('%Y%m%d'))
        if twse_resp.text != '':
            twse_pdf = pd.read_csv(StringIO(twse_resp.text.replace("=", "")), header=["證券代號" in l for l in twse_resp.text.split("\n")].index(True)-1)
            twse_pdf = twse_pdf[twse_pdf['證券代號'].str.len() == 4]
            twse_pdf['漲跌(+/-)'] = twse_pdf['漲跌(+/-)'].map(lambda x: 1 if x=='+' else -1)
            twse_pdf['漲跌價差'] = twse_pdf['漲跌價差']*twse_pdf['漲跌(+/-)']
            twse_pdf = twse_pdf.rename(columns={'證券代號':'股票代號', '證券名稱':'名稱'})
            twse_pdf = twse_pdf[header_list]
        else:
            dt = dt + timedelta(days=1)
            sleep(randint(3, 5))
            continue
        
        pdf = tpex_pdf.append(twse_pdf)
        pdf = strToFloat(pdf, ['收盤價', '開盤價', '漲跌價差', '最高價', '最低價', '成交股數', '成交金額', '成交筆數'])
        pdf['前日收盤價'] = pdf['收盤價']-pdf['漲跌價差']
        pdf['漲跌幅'] = pdf['漲跌價差']/pdf['前日收盤價']*100
        pdf = pdf.round(2)

        for row in pdf.iterrows():
            data = row[1]
            stock_no = data['股票代號']
            volume = data['成交股數']
            transaction = data['成交筆數']
            turnover = data['成交金額']
            close_price = data['收盤價']
            open_price = data['開盤價']
            high_price = data['最高價']
            low_price = data['最低價']
            change_price = data['漲跌價差']
            change_ratio = data['漲跌幅']
            row = "{},{},{},{},{},{},{},{},{},{},{}\n".format(stock_no, dt.strftime('%Y-%m-%d'), volume, turnover, 
                                                   open_price, high_price, low_price, close_price,
                                                   change_price, change_ratio, transaction)
            file_path = output_path + os.sep + stock_no + '.csv'
            
            if os.path.exists(file_path):
                file = open(file_path, 'a')
                file.write(row)
            else:
                file = open(file_path, 'w')
                file.write(header_str)
                file.write(row)
            file.close()
    dt = dt + timedelta(days=1)
    sleep(randint(1, 4))

2020/08/14


In [6]:
import smtplib

In [11]:
smtp=smtplib.SMTP('smtp.gmail.com', 587)
smtp.ehlo()
smtp.starttls()
smtp.login('b86681718@gmail.com','gjfsqhwfhrqcuene')
from_addr='b86681718@gmail.com'
to_addr="ttfangb@tsmc.com"
msg="Subject:Hello World\nHello World!"
status=smtp.sendmail(from_addr, to_addr, msg)#加密文件，避免私密信息被截取
if status=={}:
    print("郵件傳送成功!")
else:
    print("郵件傳送失敗!")
smtp.quit()

郵件傳送成功!


(221, b'2.0.0 closing connection m9sm85288pjs.18 - gsmtp')

In [ ]:
mime=MIMEText("", "plain", "utf-8") #撰寫內文內容，以及指定格式為plain，語言為中文
mime["Subject"]="BBand Alert"
mime["From"]="b86681718@gmail.com"
mime["To"]="a86681718@gmail.com"
msg=mime.as_string() #將msg將text轉成str

In [4]:
twse_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=%s&type=ALL'
twse_resp = requests.get(twse_url % '20200903')
header_list = ['股票代號', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

twse_pdf = pd.read_csv(StringIO(twse_resp.text.replace("=", "")), header=["證券代號" in l for l in twse_resp.text.split("\n")].index(True)-1)
twse_pdf = twse_pdf[twse_pdf['證券代號'].str.len() == 4]
twse_pdf['漲跌(+/-)'] = twse_pdf['漲跌(+/-)'].map(lambda x: 1 if x=='+' else -1)
twse_pdf['漲跌價差'] = twse_pdf['漲跌價差']*twse_pdf['漲跌(+/-)']
twse_pdf = twse_pdf.rename(columns={'證券代號':'股票代號', '證券名稱':'名稱'})
twse_pdf = twse_pdf[header_list]
twse_pdf

,股票代號,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,0050,"5,129,198","532,334,317",103.70,104.30,103.20,103.30,0.30,"2,346"
1,0051,"44,470","1,810,754",40.60,40.85,40.60,40.60,0.25,41
2,0052,"341,613","30,771,132",90.10,90.50,89.55,89.70,0.40,194
3,0053,"4,474","224,844",50.30,50.40,49.95,49.95,0.50,6
4,0054,"10,000","255,200",25.50,25.58,25.50,25.53,-0.00,5
...,...,...,...,...,...,...,...,...,...
18093,9944,"533,125","11,991,325",22.80,22.85,22.30,22.70,-0.10,337
18094,9945,"4,480,124","187,403,728",41.75,42.20,41.50,41.80,0.35,"2,328"
18095,9946,"453,558","7,700,136",17.05,17.30,16.90,17.00,-0.00,78
18096,9955,"211,200","4,019,690",19.10,19.20,18.85,18.95,-0.10,119


In [10]:
twse_pdf[twse_pdf['成交股數']=='0']

,股票代號,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
17224,1475,0,0,--,--,--,--,-0.0,0
18041,8497,0,0,--,--,--,--,-0.0,0


In [11]:
tpex_url = 'https://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_result.php?l=zh-tw&d=%s'
tpex_header = ['股票代號', '名稱', '收盤價', '漲跌價差', '開盤價', '最高價', '最低價', '均價', '成交股數', '成交金額', 
               '成交筆數', '最後買價', '最後買量', '最後賣價', '最後賣量', '發行股數', '次日參考價', '次日漲停價', '次日跌停價']
tpex_resp = requests.get(tpex_url % '109/09/04')
json_obj = json.loads(tpex_resp.text)
aaData = json_obj['aaData']
tpex_pdf = pd.DataFrame(aaData, columns=tpex_header)
tpex_pdf = tpex_pdf[tpex_pdf['股票代號'].str.len() == 4]
tpex_pdf = tpex_pdf[header_list]
tpex_pdf.sort_values('成交筆數')

,股票代號,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
383,4419,0,0,---,---,---,---,---,0
6134,8917,0,0,---,---,---,---,---,0
6135,8921,0,0,---,---,---,---,---,0
375,4304,0,0,---,---,---,---,---,0
151,2724,0,0,---,---,---,---,---,0
...,...,...,...,...,...,...,...,...,...
446,4931,"144,020","3,219,151",22.60,22.60,22.05,22.50,-0.45,97
622,6180,"1,193,150","79,007,420",65.20,67.50,65.20,67.40,+0.70,975
736,6594,"1,467,467","54,194,251",35.10,39.30,34.00,39.30,+3.45,978
275,3516,"264,751","3,667,263",13.90,14.00,13.70,13.85,-0.15,98


In [8]:
tmp = strToFloat(tpex_pdf, ['收盤價', '開盤價', '漲跌價差', '最高價', '最低價', '成交股數', '成交金額', '成交筆數'])
tmp[tmp['股票代號']=='6417']

,股票代號,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
680,6417,50000.0,3055900.0,60.6,62.6,60.4,61.8,NaN,27.0
